In [14]:
from IPython.display import Image, display
import os
import cv2
from PIL import Image as PILImage

# Variáveis - contador de imagens
contador = 0
total_imagens = 0

# Caminho da pasta
caminho = 'Dataset_Mosquitos\Aedes'

extensoes = ['.jpg', '.jpeg']

# Detecção de bolas brancas com bordas pretas e desenho de quadrados
def bolinhas_brancas(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blurred, threshold1=50, threshold2=100)
    contornos, _ = cv2.findContours(edges.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    #image_with_boxes = image.copy()  # Cria uma cópia da imagem para desenhar retângulos

    for contorno in contornos:
        area = cv2.contourArea(contorno)
        epsilon = 0.05 * cv2.arcLength(contorno, True)
        approx = cv2.approxPolyDP(contorno, epsilon, True)

        if area > 90 and len(approx) >= 6 and len(approx) <= 100:
            image_with_boxes = image.copy()  # Cria uma cópia da imagem para desenhar retângulos
            x, y, w, h = cv2.boundingRect(contorno)
            cv2.rectangle(image_with_boxes, (x, y), (x + w, y + h), (0, 255, 0), 2)  # Desenha um retângulo verde

            return image_with_boxes

# Análise das imagens
for filename in os.listdir(caminho):
    if any(filename.lower().endswith(ext) for ext in extensoes):
        image_path = os.path.join(caminho, filename)
        image = cv2.imread(image_path)

        # Contabilizar imagens
        total_imagens += 1

        image_with_detected_balls = bolinhas_brancas(image)
        if image_with_detected_balls is not None:
            contador += 1  # imagens detectadas corretamente
            # display(PILImage.fromarray(image_with_detected_balls))

# Calcular a acurácia
accuracy = (contador * 100) / total_imagens if total_imagens > 0 else 0

print("ANÁLISE COMPLETA DAS IMAGENS:\n")
print("TOTAL DE IMAGENS ANALISADAS:", total_imagens, "\n")
print("IMAGENS CAPTADAS DE MOSQUITOS DA DENGUE:", contador, "\n")
print("Acurácia: {:.2f}%".format(accuracy))


ANÁLISE COMPLETA DAS IMAGENS:

TOTAL DE IMAGENS ANALISADAS: 43 

IMAGENS CAPTADAS DE MOSQUITOS DA DENGUE: 13 

Acurácia: 30.23%


In [16]:
import os
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt

# Variáveis - contador de imagens
contador, total_imagens = 0, 0

# Load the template image
template = Image.open('Templates/template1.png')
# Convert the template image to grayscale
templateGray = template.convert('L')
templateGrayArray = np.array(templateGray)

# Display the template image using matplotlib
""" plt.imshow(templateGrayArray, cmap='gray')
plt.title("Template Image")
plt.axis('off')
plt.show() """

# Define the directory containing the target images

target_directory = 'Dataset_Mosquitos/Aedes'
#target_directory = 'Dataset_Mosquitos/Outros'

# Iterate through the files in the directory
for filename in os.listdir(target_directory):
    if filename.endswith(".jpg"):  # Check if the file is a JPEG image
        # Load the target image
        target_path = os.path.join(target_directory, filename)
        image = Image.open(target_path)
        matched_image = cv2.imread(target_path)
        total_imagens += 1

        # Convert the target image to grayscale
        imageGray = image.convert('L')
        imageGrayArray = np.array(imageGray)

        # Ensure the template is smaller or equal in size to the target image
        if templateGrayArray.shape[0] <= imageGrayArray.shape[0] and templateGrayArray.shape[1] <= imageGrayArray.shape[1]:
            # Find template
            result = cv2.matchTemplate(imageGrayArray, templateGrayArray, cv2.TM_CCOEFF)
            _, max_val, _, max_loc = cv2.minMaxLoc(result)
            if max_val > 358000:
                contador += 1
"""                 cv2.rectangle(matched_image, max_loc, (max_loc[0] + templateGrayArray.shape[0], max_loc[1] + templateGrayArray.shape[1]), (0, 255, 0), 4)
                plt.title(f"Match found in {filename} at location: {max_loc}")
                plt.imshow(cv2.cvtColor(matched_image, cv2.COLOR_BGR2RGB))
                plt.axis('off')
                plt.show()
            else:
                plt.title(f"Match NOT found in {filename}")
                plt.imshow(cv2.cvtColor(matched_image, cv2.COLOR_BGR2RGB))
                plt.axis('off')
                plt.show() """

# Calcular os acertos
acertos = (contador * 100) / total_imagens if total_imagens > 0 else 0

print("ANÁLISE COMPLETA DAS IMAGENS:\n")
print("TOTAL DE IMAGENS ANALISADAS:", total_imagens, "\n")
print("IMAGENS CAPTADAS DE MOSQUITOS DA DENGUE:", contador, "\n")
print("Acertos: {:.2f}%".format(acertos))

ANÁLISE COMPLETA DAS IMAGENS:

TOTAL DE IMAGENS ANALISADAS: 34 

IMAGENS CAPTADAS DE MOSQUITOS DA DENGUE: 32 

Acertos: 94.12%
